# Práctica 2: Regresión logística {-}
Álvar Domingo Fernández y Pablo Jurado López

---

### Preparación inicial {-}
A continuación se importan todas las librerías que serán utilizadas en esta práctica y se indica el método que se utilizará para cargar los datos:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.parsers import read_csv
import scipy.optimize as opt
from sklearn.preprocessing import PolynomialFeatures


def carga_csv(file_name):
    return read_csv(file_name, header=None).to_numpy().astype(float)

## 1.1 - Visualización de los datos {-}
A partir de un fichero con los datos, hemos utilizado la libreria matplot para representarlos en una gráfica, representando de distinta forma los puntos que representan ser admitido o no admitido.

In [ ]:
data = carga_csv('ex2data1.csv')
X = data[:, :-1]
Y = data[:, -1]
# Obtiene un vector con los índices de los ejemplos positivos
pos = np.where(Y == 1)
neg = np.where(Y == 0)

# Dibuja los ejemplos positivos
plt.figure(0)
plt.scatter(X[pos, 0], X[pos, 1], marker='+', c='k', label='Admitted')
plt.scatter(X[neg, 0], X[neg, 1], marker='o', c='g', label='Not admitted')
plt.xlabel('Exam 1 score')
plt.ylabel('Exam 2 score')
plt.legend()
plt.savefig('dataGraph1')

![Representación inicial de los datos](dataGraph1.png)

## 1.2 - Función sigmoide {-}
Se ha implementado una función sigmoide, definida por la siguiente fórmula:
$$g(z) = \frac{1}{1+e^{-z}}$$

In [ ]:
def sigmoide(target):
    result = 1 / (1 + np.exp(-target))
    return result

## 1.3 - Cálculo de la función de coste y su gradiente {-}
Se ha implementado la función de coste en regresión logística, que en forma vectorizada viene dada por la siguiente fórmula:
$$J(\theta) = \frac{1}{m}\left(-(\log(g(X\theta))^T y - (\log(1 -g(X\theta))^T (1-y)\right)$$

In [ ]:
def coste(theta, X, Y):
    m = np.shape(X)[0]
    H = sigmoide(np.matmul(X, theta))
    return (np.dot(np.transpose(np.log(H)), Y) + np.dot(np.transpose(np.log(1-H)), (1-Y))) / -m

También se ha implementado la función que obtiene el gradiente de la función de coste, que viene definida en su forma vectorizada por la siguiente fórmula:
$$\frac{\delta J(\theta)}{\delta \theta} = \frac{1}{m} X^T \left(g(X \theta) - y\right)$$

In [ ]:
def gradiente(theta, X, Y):
    m = np.shape(X)[0]
    H = sigmoide(np.matmul(X, theta))
    return (np.matmul(X.T, H - Y)) / m

## 1.4 - Cálculo del valor óptimo de los parámetros {-}
Se ha utilizado la función scipy.optimize.fmin_tnc de SciPy para hallar los parámetros $\theta$ que minimizan la función de coste para la regresión del apartado anterior:

In [ ]:
result = opt.fmin_tnc(func=coste, x0=theta, fprime=gradiente, args=(X, Y))
theta_opt = result[0]
print(coste(theta_opt, X, Y))

El valor de la función de coste ha sido en este caso de aproximadamente 0.69

A continuación se han representado los resultados en una gráfica que, gracias a este cálculo, también dibuja la frontera de decisión:

In [ ]:
# Dibuja los ejemplos positivos
plt.figure(0)
x1_min, x1_max = X[:, 1].min(), X[:, 1].max()
x2_min, x2_max = X[:, 1].min(), X[:, 2].max()

xx1, xx2 = np.meshgrid(np.linspace(x1_min, x1_max),
                       np.linspace(x2_min, x2_max))

h = sigmoide(np.c_[np.ones((xx1.ravel().shape[0], 1)),
                   xx1.ravel(),
                   xx2.ravel()].dot(theta_opt))
h = h.reshape(xx1.shape)

plt.contour(xx1, xx2, h, [0.5], linewidths=1, colors='b')
plt.savefig('dataGraph1line')
plt.close()

![Representación de los datos y su frontera de decisión](dataGraph1line.png)

## 1.5 - Evaluación de la regresión logística {-}
Se ha implementado una función que da como resultado el porcentaje de los casos de entrenamiento que se han clasificado de manera correcta, es decir, aquellos que han quedado en el lado correcto de la frontera de decisión en la imagen anterior:

In [ ]:
prediccion = sigmoide(np.matmul(X, theta_opt))
correctos = np.mean((prediccion >= 0.5) == Y)
print(correctos)

El resultado en este caso ha sido de un 89%

## 2 - Regresión logística regularizada